In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 500)

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [3]:
from utils import UtilsKy
from analyzer import AnalyzerPrediction

In [4]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

In [8]:
import joblib

In [10]:
model_name = 'pytorch_kyw3_30-09-001_2021-01-25'

## We need to modlel class(MLP) For load modlel

In [13]:
from pytorch_models import *

In [14]:
m_config = joblib.load(model_name)

In [27]:
model = m_config.get('profile')

In [15]:
def get_predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [16]:
db_test_orig = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str, encoding='cp1251')

In [17]:
def get_one_row_scaler(test, factor_list, scaler_params, replaced_values):    
    test_scaler = test.apply(pd.to_numeric, errors="coerce").copy()
    for factor in factor_list:
        key_factor_min = factor + '_min'
        key_factor_max = factor + '_max'
        
        x_min = scaler_params[key_factor_min]
        x_max = scaler_params[key_factor_max]
        
        replaced_val = replaced_values.get(factor) or replaced_values.get('default')                
        
        test_scaler[factor] = test_scaler[factor].fillna(replaced_val)
        
        test_scaler[factor] = (test_scaler[factor] - x_min) / (x_max - x_min)
        test_scaler[factor] = np.where(test_scaler[factor] > 1,  1, test_scaler[factor])
        test_scaler[factor] = np.where(test_scaler[factor] < 0 , 0, test_scaler[factor])        
    
    return test_scaler

In [18]:
scaler_params = m_config['scaler_params']
factor_list = m_config['factor_list']
m1 = m_config.get('profile')
replaced_values = m_config['replaced_values']

In [19]:
test_one_order = db_test_orig[factor_list].iloc[0:1,:]
test_one_order

,amount,bank_currency,bin,count_months_to_end_card,day_of_week,hour,is_city_resolved,is_gender_undefined,latitude,longitude,phone_2_norm
0,158.85,840,510932,19,2,00,1,1,44.1843,-88.5305,20


In [20]:
test_scaler = get_one_row_scaler(test_one_order, factor_list, scaler_params,replaced_values)

In [21]:
row = test_scaler.values[0]
row

array([0.10530354, 0.97435897, 0.01856223, 0.02159091, 0.16666667,
       0.        , 1.        , 1.        , 0.76396881, 0.23937762,
       0.2020202 ])

In [22]:
get_predict(row, m1)[0][0]

0.016264448